In [6]:
import tensorflow as tf
import pandas as pd
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import json
import gc

In [5]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00


In [48]:

df = pd.read_csv('train.csv', header=None) #files are uploaded to google colab as local system is taking time to train
df.head()

,0,1
0,BOS Sir I created a new env with python 3.7 bu...,need_help
1,BOS above problem how will it solve ?? EOS,need_help
2,BOS Hello sir is there anyone EOS,Greetings
3,BOS i think canvas login problem was due to br...,great
4,BOS Please provide me with the link to python ...,community_class_link


In [49]:
df.columns=['text','category']
df.head(10)

,text,category
0,BOS Sir I created a new env with python 3.7 bu...,need_help
1,BOS above problem how will it solve ?? EOS,need_help
2,BOS Hello sir is there anyone EOS,Greetings
3,BOS i think canvas login problem was due to br...,great
4,BOS Please provide me with the link to python ...,community_class_link
5,BOS Understood EOS,great
6,BOS Tq EOS,welocme_reply
7,BOS so ill just copy paste what I wanted to kn...,need_help
8,BOS sir any sujjestion to overcome this error?...,need_help
9,"BOS np.random.choice([&apos;Goal Area&apos;, &...",code_error


In [50]:
df['encoded'] = df['category'].astype('category').cat.codes
df.head()

,text,category,encoded
0,BOS Sir I created a new env with python 3.7 bu...,need_help,93
1,BOS above problem how will it solve ?? EOS,need_help,93
2,BOS Hello sir is there anyone EOS,Greetings,6
3,BOS i think canvas login problem was due to br...,great,77
4,BOS Please provide me with the link to python ...,community_class_link,55


In [51]:
df['category'].value_counts()

need_help                781
O                        609
Greetings                331
Thank_you                284
good_bye                 169
                        ... 
aws_deployment_error       1
assignment_matplotlib      1
file_not_found             1
aws_lex_smtp_error         1
assignment_by_github       1
Name: category, Length: 151, dtype: int64

In [52]:
df.tail()

,text,category,encoded
3509,BOS it is not one to one resume but some advic...,need_help,93
3510,BOS hi good noon EOS,Greetings,6
3511,BOS now what will year contain EOS,code_error,52
3512,BOS I have extracted it but didn&apos;t found ...,need_help,93
3513,BOS file name or extension path name EOS,need_help,93


In [19]:
train_x = df["text"].to_list()
train_y = df["encoded"].to_list()

In [20]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [21]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
validation_encodings = tokenizer(validation_texts, truncation=True, padding=True)

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(validation_encodings),
    validation_labels
))

In [25]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=151)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [26]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=30, batch_size=16, validation_data=validation_dataset.shuffle(1000).batch(16))

Epoch 1/30
174/174 [==============================] - 202s 993ms/step - loss: 2.8370 - accuracy: 0.4594 - val_loss: 2.0794 - val_accuracy: 0.5391
Epoch 2/30
174/174 [==============================] - 164s 945ms/step - loss: 1.9338 - accuracy: 0.5651 - val_loss: 1.8177 - val_accuracy: 0.5775
Epoch 3/30
174/174 [==============================] - 164s 943ms/step - loss: 1.5396 - accuracy: 0.6373 - val_loss: 1.6949 - val_accuracy: 0.6060
Epoch 4/30
174/174 [==============================] - 164s 942ms/step - loss: 1.2047 - accuracy: 0.7185 - val_loss: 1.6671 - val_accuracy: 0.6344
Epoch 5/30
174/174 [==============================] - 171s 983ms/step - loss: 0.9692 - accuracy: 0.7707 - val_loss: 1.7251 - val_accuracy: 0.6430
Epoch 6/30
174/174 [==============================] - 164s 944ms/step - loss: 0.7735 - accuracy: 0.8149 - val_loss: 1.8064 - val_accuracy: 0.6543
Epoch 7/30
174/174 [==============================] - 171s 983ms/step - loss: 0.6223 - accuracy: 0.8526 - val_loss: 1.8882 -

In [28]:
location = "/content/distil" # in google colab, downloaded it later

model.save_pretrained(location)
tokenizer.save_pretrained(location)

('/content/distil/tokenizer_config.json',
 '/content/distil/special_tokens_map.json',
 '/content/distil/vocab.txt',
 '/content/distil/added_tokens.json')

In [29]:
testing_tokenizer = DistilBertTokenizer.from_pretrained(location)
testing_model = TFDistilBertForSequenceClassification.from_pretrained(location)

Some layers from the model checkpoint at /content/distil were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/distil and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
test_text='Hi! This is Lokesh and I love AI.'
test_text

'Hi! This is Lokesh and I love AI.'

In [31]:
predict= testing_tokenizer.encode(test_text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

output= testing_model(predict)[0]

predicted_value = tf.argmax(output, axis=1).numpy()[0]
df.loc[df['encoded'] == predicted_value, 'category'].iloc[0]

'Greetings'